## Web Scraping
## Akewak Jeba

Scraping data from yelp website

In [1]:
## all imports
from IPython.display import HTML
import numpy as np
import urllib2
import bs4 #this is beautiful soup
import time
import operator
import socket
import cPickle
import re # regular expressions

from pandas import Series
import pandas as pd
from pandas import DataFrame

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_context("talk")
sns.set_style("white")

#from secret import *

In [36]:
# Fixed url for restaurant postings 
#https://www.yelp.com/search?cflt=restaurants&find_loc=Helsinki%2C+Finland
url = 'https://www.yelp.com/search?find_loc=helsinki&start=0&sortby=review_count&cflt=restaurants&attrs=GoodForMeal.breakfast,GoodForMeal.brunch,RestaurantsPriceRange2.1,RestaurantsPriceRange2.2'
# read the website
source = urllib2.urlopen(url).read()
# parse html code
bs_tree = bs4.BeautifulSoup(source)

In [37]:
restaurant_count_string = bs_tree.find("span",{"class": "pagination-results-window"}).contents[0]
restaurant_count_string = restaurant_count_string.split()[-1]
print restaurant_count_string
print("Search yielded %s hits." % (restaurant_count_string))

# note that restaurant_count so far is still a string, 
# not an integer, and the , separator prevents 
# us from just casting it to int

restaurant_count_digits = [int(d) for d in restaurant_count_string if d.isdigit()]
print restaurant_count_digits
restaurant_count = np.sum([digit*(10**exponent) for digit, exponent in 
                    zip(restaurant_count_digits[::-1], range(len(restaurant_count_digits)))])

print restaurant_count

17
Search yielded 17 hits.
[1, 7]
17


In [ ]:
num_pages = int(np.ceil(restaurant_count/10.0))

base_url = 'https://www.yelp.com/search?find_loc=helsinki&start=0&sortby=review_count&cflt=restaurants&attrs=GoodForMeal.breakfast,GoodForMeal.brunch,RestaurantsPriceRange2.1,RestaurantsPriceRange2.2'
restaurant_links = []
for i in range(30): #do range(num_pages) if you want them all
    if i%10==0:
        print num_pages-i
       # https://www.yelp.com/search?cflt=restaurants&find_loc=Helsinki%2C+Finland
    url = 'https://www.yelp.com/search?find_loc=helsinki&start=0&sortby=review_count&cflt=restaurants&attrs=GoodForMeal.breakfast,GoodForMeal.brunch,RestaurantsPriceRange2.1,RestaurantsPriceRange2.2' + str(i*10)
    html_page = urllib2.urlopen(url).read() 
    bs_tree = bs4.BeautifulSoup(html_page)
    restaurant_link_area = bs_tree.find(id = 'super-container')
    restaurant_postings = restaurant_link_area.findAll("div")
    restaurant_postings = [rp for rp in restaurant_postings if not rp.get('class') is None 
                    and ''.join(rp.get('class')) =="media-avatar"]
    restaurant_ids = [rp.get('data-hovercard-id') for rp in restaurant_postings]
    id_d = bs_tree.findAll("a",{"class": "biz-name js-analytics-click"})
    
    #for item in id_d:
   #     print item.contents[0].text.split(' ', 1)[0]
    for sr in bs_tree.find_all('div', attrs={"class":u"search-result"}):
        
        
        
        name = bs_tree.find('a', attrs={"class":u"biz-name"})['href']
        #catagory = list(sr.find_all('span',attrs={"class":u"category-str-list"})[0].get_text().split())#[1:20] 
        catagory = list(sr.find_all('span',attrs={"class":u"category-str-list"})[0].get_text().split())[:20]           
        if (catagory[0] or catagory[1] or catagory[2] or catagory[3] or catagory[4] or catagory[5] or catagory[6]or catagory[7] or catagory[8]or catagory[9] or catagory[10]) == 'Breakfast':
            
            restaurant_links.append(base_url + name)
        if (catagory[0] or catagory[1] or catagory[2] or catagory[3] or catagory[4] or catagory[5] or catagory[6]  or catagory[7] or catagory[8]or catagory[9] or catagory[10]) == 'brunch':
            restaurant_links.append(base_url + name)
        
        print catagory
         
        
            
        

In [40]:
print "We found a lot of restaurants: ", len(restaurant_links)

We found a lot of restaurants:  300


In [41]:
with open('restaurant_scraped_links.pkl', 'wb') as f:
    cPickle.dump(restaurant_links, f)

In [33]:
with open('restaurant_scraped_links.pkl', 'r') as f:
    restaurant_links = cPickle.load(f)

In [35]:
restaurant_name_list =[]
restaurant_review_list=[]
rating_list =[]


for url in restaurant_links:
    url_page = urllib2.urlopen(url).read()
    soup = bs4.BeautifulSoup(url_page)
    #restaurant_name = soup.find("span",{"class": "indexed-biz-name"})
    restaurant_name = soup.find('a', attrs={"class":u"biz-name js-analytics-click"}).get_text()
    
    
    restaurant_review = soup.find("span",{"class": "review-count rating-qualifier"}).get_text()
    #ratings = soup.find("div",{"class": "i-stars i-stars--large-4-half rating-very-large"})['title']
    review_count_digits = [int(d) for d in restaurant_review if d.isdigit()]
    review_count = np.sum([digit*(10**exponent) for digit, exponent in 
                    zip(review_count_digits[::-1], range(len(review_count_digits)))])
    
    rating_count_digits = [int(d) for d in ratings if d.isdigit()]
    #print rating_count_digits
    rating_count = np.sum([digit*(10**exponent) for digit, exponent in 
                    zip(rating_count_digits[::-1], range(len(rating_count_digits)))])
    
    rate_rate = (rating_count)/float(10)
    
    restaurant_name_list.append(restaurant_name)
    restaurant_review_list.append(review_count)
    rating_list.append(rate_rate)
print restaurant_name_list
print restaurant_review_list
print rating_list

    #print restaurant_name,restaurant_review,ratings
    #print type(restaurant_name)
data = [x.split(',') for x in restaurant_name_list]
print data
name = pd.DataFrame(data, columns=data[0])
print name
review = pd.DataFrame(restaurant_review_list)
print review
rate = pd.DataFrame(rating_list)
print rate
#frames = [name, review, rate]
result = pd.concat([name, review,rate], axis=1)
print result

[u'Pompier', u'Kuu', u'Bites Vallila', u'Ravintola Talin Kartano']
[13, 23, 2, 3]
[4.5, 4.5, 4.5, 4.5]
[[u'Pompier'], [u'Kuu'], [u'Bites Vallila'], [u'Ravintola Talin Kartano']]
                   Pompier
0                  Pompier
1                      Kuu
2            Bites Vallila
3  Ravintola Talin Kartano
    0
0  13
1  23
2   2
3   3
     0
0  4.5
1  4.5
2  4.5
3  4.5
                   Pompier   0    0
0                  Pompier  13  4.5
1                      Kuu  23  4.5
2            Bites Vallila   2  4.5
3  Ravintola Talin Kartano   3  4.5
